<a href="https://colab.research.google.com/github/ivmarchuk/Garch_AAPL/blob/main/Garch_APPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/RJT1990/pyflux

In [ ]:
! pip install pyflux

In [ ]:
!pip install yfinance

In [ ]:
!pip install arch

In [ ]:
import numpy as np
import pandas as pd

from pandas_datareader import DataReader 
import pandas_datareader.data as web

from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt

import pyflux as pf
import yfinance as yf

from arch import arch_model
from statsmodels.tsa.arima_model import ARIMA

import sys

In [ ]:
assets=['AAPL']
df=pd.DataFrame()

for i in assets:
  df[i]=yf.download(i,'2015-08-01','2022-08-01')['Close']
  plt.plot(df[i],label=i)
  
plt.legend()
plt.show()
returns=df.pct_change() 

In [ ]:
df['AAPL']=yf.download('AAPL','2015-08-01')['Close']

In [ ]:
returns_1 = pd.DataFrame(np.diff(np.log(df['AAPL'].values)))
print(returns_1)

returns_1.index=df.index.values[1:df.index.values.shape[0]]

plt.plot(df)
plt.figure(figsize=(15,5));
plt.plot(returns.index,returns)
plt.ylabel('Returns')
plt.title('AAPL Returns')

In [ ]:
data_1=returns_1
train = data_1[1:round(len(returns)/2)+2]
test = data_1[round(len(returns)/2):]

print(train)

In [ ]:
am = arch_model(data_1, vol='Garch', p=1, q=1)
index = data_1.index
start_loc = 0
end_loc = np.where(index >= "2018-5-17")[0].min()
forecasts = {}

for i in range(880):
 sys.stdout.write(".")
 sys.stdout.flush()
 res = am.fit(first_obs=i, last_obs=i + end_loc, disp="off")
 temp = res.forecast(horizon=1, reindex=False).variance
 fcast = temp.iloc[0]
 forecasts[fcast.name] = fcast
 pred=pd.DataFrame(forecasts).T
 
print()
print(pred)

In [ ]:
print(res.summary())

In [ ]:
var2 = pd.concat([test], axis=1)
var2['Predictions'] = np.array(pred)
var2.columns = ['Test','Prediction']
print(var2)

In [ ]:
assets=['AAPL']
df=pd.DataFrame()
for i in assets:
  df[i]=yf.download(i,'2015-01-03')['Close']
  plt.plot(df[i],label=i)
plt.legend()
plt.show()
returns=df.pct_change()

In [ ]:
var = train**2
print(var)

In [ ]:
trivial=[]

for i in range(848):
  trivial.append(data_1[i:(848+i)].var().values[0])
trivial=pd.DataFrame(trivial)
print(trivial)

In [ ]:
var2 = test**2
print(var2)

In [ ]:
var2 = pd.concat([var2], axis=1)
print(var2)

In [ ]:
var2.columns = ['TEST']
print(var2)

In [ ]:
var2.columns = ['Error']
print(var2)

In [ ]:
tr=var2['TEST']**2
tr.mean()
print (tr.mean())
# trivial models error

In [ ]:
var2['Predictions'] = np.array(pred) 
print(var2)

In [ ]:
var2['TEST2'] = var2['TEST'] - var2['Predictions']
print(var2)

In [ ]:
preder = var2['Error2']**2
preder.mean()
print(preder.mean())
# models mistake

In [ ]:
result = tr.mean()-preder.mean()
print(result)